In [1]:
import pandana
import osmnx as ox
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import geoalchemy2

C:\Users\apikuleva\Anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\apikuleva\Anaconda3\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.

In [2]:
# функция для чтения того, что получили в запросе
def select_pg(sql, geom):
    return gpd.read_postgis(sql, engine, geom_col = geom, crs = 3857)

engine = create_engine('postgresql+psycopg2://kb_geo:kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h@\
db03.cluster.strlk.ru/kb_graph')

sql = "SELECT geom, lng, lat, id, in_out, category, owner_id, date, city, home_town FROM public.photo_main "
poi = select_pg(sql, 'geom')
# poi = gpd.GeoDataFrame(poi, geometry = poi['geom'], crs  = 3857)

In [7]:
edge_graph_moscow_spb = "SELECT * FROM public.edge_graph_moscow_spb "
node_graph_moscow_spb = "SELECT * FROM public.node_graph_moscow_spb "

node_graph = select_pg(node_graph_moscow_spb, 'geom')
edge_graph = select_pg(edge_graph_moscow_spb, 'geometry')

In [11]:
node_graph['lng']  = node_graph['geom'].x
node_graph['lat']  = node_graph['geom'].y

In [21]:
node_graph = node_graph.rename(columns={"lng": "x", "lat": "y"})

In [25]:
edge_graph

,geometry,u,v,reversed,oneway,lenght,highway,speed
0,"LINESTRING (3149489.372 7931200.771, 3149457.6...",123782,123781,True,B,72.504722,steps,NaN
1,"LINESTRING (3149279.000 7929659.922, 3149279.0...",128843,128840,True,B,17.348745,footway,5.0
2,"LINESTRING (3149302.834 7929656.264, 3149336.4...",129017,129018,False,B,33.652116,footway,5.0
3,"LINESTRING (3149253.709 7929656.076, 3149253.7...",128839,128842,False,B,28.957175,footway,5.0
4,"LINESTRING (3149350.891 7929656.202, 3149350.8...",122917,129018,True,B,19.752944,footway,5.0
...,...,...,...,...,...,...,...,...
6734214,"LINESTRING (5332887.166 7689110.654, 5332864.1...",2486582,2484952,False,B,67.915910,residential,40.0
6734215,"LINESTRING (5332109.310 7689313.942, 5332201.3...",2487545,2484927,True,B,278.434294,residential,40.0
6734216,"LINESTRING (5331841.008 7689030.506, 5331796.2...",2484471,2484462,False,B,140.698796,residential,40.0
6734217,"LINESTRING (5331400.450 7688530.688, 5331319.3...",2484482,2485493,True,B,85.545158,residential,40.0


In [26]:
node_graph = node_graph.set_index('osmid')


SyntaxError: EOL while scanning string literal (3761004289.py, line 3)

In [28]:
edge_graph['key'] = 0
edge_graph = edge_graph.set_index(['u', 'v', 'key'])

In [29]:
G = ox.utils_graph.graph_from_gdfs(node_graph, edge_graph, graph_attrs=None)

In [ ]:
G = ox.project_graph(G, 3857)

In [ ]:
geom_from_graph = poi['geom'].unary_union.convex_hull.buffer(10000)
geom_from_graph_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:3857', geometry=[geom_from_graph])
geom_from_graph_gdf = geom_from_graph_gdf.to_crs(4326)
G = ox.graph_from_polygon(geom_from_graph_gdf['geometry'].iloc[0], network_type='all', simplify=True, retain_all=False,
                          truncate_by_edge=True, clean_periphery=True)


In [ ]:
G = ox.graph_from_polygon(gdf['geometry'].iloc[0], network_type=network_type, simplify=True, retain_all=False,
                              truncate_by_edge=True, clean_periphery=True, custom_filter = cf)

In [81]:
# poi = poi.
poi = poi.sort_values(['owner_id','date_time'])
poi

,lng,lat,id,owner_id,date_time,nx_node,dist,lat_next,lng_next,distances
25402,3.156829e+06,7.926032e+06,47099067,0,2022-01-01 10:48:54+00:00,6192,24.733015,7.926036e+06,3.156832e+06,1360.0
25422,3.156832e+06,7.926036e+06,59459690,0,2022-01-01 10:48:54+00:00,6192,29.915627,7.928764e+06,3.154818e+06,0.0
3609,3.154818e+06,7.928764e+06,7935681,0,2022-01-01 11:26:52+00:00,2018,20.482439,7.928181e+06,3.154899e+06,0.0
43451,3.154899e+06,7.928181e+06,3668926,0,2022-01-01 11:32:49+00:00,4630,29.961500,7.928703e+06,3.154797e+06,0.0
28517,3.154797e+06,7.928703e+06,64134100,0,2022-01-01 11:37:56+00:00,2042,6.283379,7.928527e+06,3.147321e+06,1120.0
...,...,...,...,...,...,...,...,...,...,...
38777,3.155469e+06,7.929904e+06,39382898,769030369,2022-12-11 20:09:58+00:00,9538,59.387649,7.927804e+06,3.156331e+06,520.0
50760,3.156331e+06,7.927804e+06,4574474,769832502,2022-12-16 14:08:39+00:00,4670,21.910938,7.933070e+06,3.155775e+06,0.0
5148,3.155775e+06,7.933070e+06,51656348,771736503,2022-12-22 08:08:48+00:00,5427,32.979514,7.929789e+06,3.155515e+06,2000.0
21048,3.155515e+06,7.929789e+06,81222998,771937919,2022-12-27 15:44:57+00:00,719,18.307238,7.929646e+06,3.154204e+06,0.0


In [2]:
G = ox.graph_from_place('Псков', network_type = 'all',simplify=True, retain_all=False,
                                truncate_by_edge=True, clean_periphery=True, custom_filter = None)
G = ox.projection.project_graph(G, to_crs=3857)

#упрощение для получения реальных узлов пересечений
G_simplified = ox.simplification.consolidate_intersections(
    # Graph to simplify
    G,
    # buffer around each node (project the graph beforehand)
    tolerance=5,
    # Get result as graph (False to get nodes only as gdf)
    rebuild_graph=True,
    # no dead ends
    dead_ends=False,
    # Reconnect (False to get intersections only)
    reconnect_edges=True
)

# Max time to walk in minutes (no routing to nodes further than this)
walk_time = 15

# Walking speed
walk_speed = 40

# Set a uniform walking speed on every edge
for u, v, data in G_simplified.edges(data=True):
    data['speed_kph'] = walk_speed
graph = ox.add_edge_travel_times(G_simplified)

graph = ox.distance.add_edge_lengths(G_simplified, precision=3, edges=None)

#  Extract node/edge GeoDataFrames, retaining only necessary columns (for pandana)
nodes = ox.graph_to_gdfs(G_simplified, edges=False).reset_index()
edges = ox.graph_to_gdfs(G_simplified, nodes=False).reset_index()




In [ ]:
# create network with pandana
graph = pandana.Network(nodes['x'], nodes['y'], 
                          edges['u'], edges['v'], edges[['lenght']])

In [6]:
# Max time to walk in minutes (no routing to nodes further than this)
walk_time = 15

# Walking speed
walk_speed = 40

# Set a uniform walking speed on every edge
for u, v, data in G_simplified.edges(data=True):
    data['speed_kph'] = walk_speed
graph = ox.add_edge_travel_times(G_simplified)

graph = ox.distance.add_edge_lengths(graph, precision=3, edges=None)

In [35]:
#нашли блиажайший узел к каждой точке
poi['nx_node'], poi['dist'] = ox.distance.nearest_nodes(G_simplified, poi.lng,poi.lat, return_dist=True)

In [36]:
poi['lat_next'] = poi.lat.shift(-1)
poi['lng_next'] = poi.lng.shift(-1)

In [37]:
poi.drop(poi.tail(1).index,inplace=True) # drop last n rows 

In [38]:
origin_nodes = graph.get_node_ids(poi.lng, poi.lat).values
dests_nodes = graph.get_node_ids(poi.lng_next, poi.lat_next).values

In [40]:
#пример нахождения ближайшего пути в pandana
poi['distances'] = pd.Series(graph.shortest_path_lengths(origin_nodes, dests_nodes))

In [55]:
# ближайший узел для каждой точки
nodes_nearest = graph.get_node_ids(poi.lng, poi.lat).tolist()

In [61]:
nodes_list_2 = []
i_list_2 = []

for i in nodes_nearest:
    #кратчайший путь между точками
    k = graph.shortest_path(nodes_nearest[i], nodes_nearest[i+1])
# # print(k)
    nodes_list_2.append(k)
    i_list_2.append(i)
    

In [82]:
path = pd.DataFrame(list(zip(nodes_list_2, i_list_2)))

In [74]:
path = path.explode(0)
path[0] = pd.to_numeric(path[0], downcast='integer')

In [86]:
path.to_sql('path_pskov', engine)

951